In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import sys
sys.path.append('Biopredict')

In [2]:
from Prediction import predict
from Featurizer import featurizer
from biopred import biopred
from ADpredict import Applicability_domain_LOF

activity_col = 'pChEMBL Value'
smile_col = 'SMILES'
task_type = 'C'
target_thresh = 5
var_thresh = 0.05
scoring = 'f1'
ID_col = 'ID'
target_dir = os.getcwd()+ '/Target/'
target_type  = ['A549', 'MDAMB231', 'Tubulin']
raw_ad_dir = os.getcwd() +'/Data/Raw_ad'
ad_dir = os.getcwd()+'/Data/AD'

In [22]:
df = pd.read_csv(raw_ad_dir+'/ad_A549.csv')

In [25]:
df

,pChEMBL Value,Canomicalsmiles
0,5.47,COc1cc([C@@H]2c3cc4c(cc3C[C@H]3COC(=O)[C@@H]32...
1,4.99,C/C=C/c1ccc(Nc2ncnc3cc(OC)c(OCCCN4CCOCC4)cc23)cc1
2,4.97,CN1CCCN(c2c3c4ccccc4nc-3oc3ccc(Cl)cc23)CC1
3,4.70,O=C(Nc1cccc(F)c1)Nc1nc2ccccc2[nH]1
4,4.55,CCCCn1c2cc(OCCCc3ccccc3)ccc2c2ccnc(C)c21
...,...,...
8974,5.23,O=C(Nc1cccc(S(=O)(=O)c2cccc(NC(=O)C(Cl)Cl)c2)c...
8975,5.38,O=C1c2cccnc2C(=O)c2c1c(C(=O)NCCCN1CCOCC1)c1ccc...
8976,5.06,NS(=O)(=O)c1ccc(-n2c(SCC(=O)Nc3ccc(Br)cc3)nc3c...
8977,5.33,Cc1cc(C)c(CNc2nc(-c3ccc(N4CCN(C)CC4)cc3)nc3c2n...


In [32]:
for i in target_type:
    name = raw_ad_dir+'/ad_'+i+'.csv'
    print(name)
    df = pd.read_csv(name)
    df['ID'] = df.index.astype('object')
    feature = featurizer(data = df, ID_col= 'ID', smiles_col = 'Canomicalsmiles', type_fp ='RDK7')
    data = feature.fit()
    data.drop(['ID'], axis=1, inplace = True)
    data = data.iloc[:,:4097]
    data.to_csv(ad_dir+'/ad_'+i+'.csv')
    display(data.head(2))

/home/labhhc/Documents/Workspace/D18/SonD18/Tubulin_virtual_screening/Data/Raw_ad/ad_A549.csv


,pChEMBL Value,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,1,1,1,0,0,0,1,0,1,...,1,1,0,0,0,1,1,1,1,0
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


/home/labhhc/Documents/Workspace/D18/SonD18/Tubulin_virtual_screening/Data/Raw_ad/ad_MDAMB231.csv


,pChEMBL Value,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/labhhc/Documents/Workspace/D18/SonD18/Tubulin_virtual_screening/Data/Raw_ad/ad_Tubulin.csv


,pChEMBL Value,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,1,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,1,1,0,1
1,0,0,1,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0


# 1. SMILES

In [13]:
input_smiles = 'COc1cc2c(c(OC)c1OC)-c1ccc(OC)c(=O)cc1[C@@H](NC(C)=O)CC2'
input_ID = 'colchicine'
dict_df = {'ID':[input_ID], 'SMILES':[input_smiles]}

df = pd.DataFrame.from_dict(dict_df)

In [14]:
feature = featurizer(data = df, ID_col= 'ID', smiles_col = 'SMILES', type_fp ='RDK7')
data = feature.fit()

ad = Applicability_domain_LOF(data=data, ad_dir=ad_dir)
ad.fit()

final = biopred(data=data, ID_col ='ID', activity_col = 'pChEMBL Value', target_dir = target_dir)

final['Cytotoxic'] = final['Pred_A549'] + final['Pred_MDAMB231']
final['Cytotoxic'][final['Cytotoxic'] >0] =1
final['final_pre'] = final['Cytotoxic']+ final['Pred_Tubulin']
print('Number data screen:', data.shape[0])
print('Number data cytotoxic predict', final[final['Cytotoxic']>0].shape[0])
print('Number data Tubulin predict', final[final['final_pre']>1].shape[0])

df.index = df['ID']
AD_pred = ad.data
AD_pred.index = AD_pred['ID']

data_pred = pd.concat([df[['ID','SMILES']], final.drop(final.columns[:2], axis =1),AD_pred.iloc[:,-3:]], axis =1)

sort_pred = data_pred.sort_values(by='Proba_Tubulin',ascending = False)
sort_pred.head()

/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


ValueError: X has 4096 features, but LocalOutlierFactor is expecting 8192 features as input.

# 2. Database search

In [33]:
df = pd.read_csv('Data/Test.csv')
df = df[['ID','SMILES']]
df.head()

,ID,SMILES
0,CHEMBL107,COc1cc2c(c(OC)c1OC)-c1ccc(OC)c(=O)cc1[C@@H](NC...
1,CHEMBL67,COc1ccc(/C=C\c2cc(OC)c(OC)c(OC)c2)cc1O
2,CHEMBL159,CC[C@]1(O)C[C@@H]2CN(CCc3c([nH]c4ccccc34)[C@@]...
3,CHEMBL90555,CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](...
4,CHEMBL553025,CCC1=C[C@@H]2CN(C1)Cc1c([nH]c3ccccc13)[C@@](C(...


In [34]:
feature = featurizer(data = df, ID_col= 'ID', smiles_col = 'SMILES', type_fp ='RDK7')
data = feature.fit()

ad = Applicability_domain_LOF(data=data, ad_dir=ad_dir)
ad.fit()

final = biopred(data=data, ID_col ='ID', activity_col = 'pChEMBL Value', target_dir = target_dir)

final['Cytotoxic'] = final['Pred_A549'] + final['Pred_MDAMB231']
final['Cytotoxic'][final['Cytotoxic'] >0] =1
final['final_pre'] = final['Cytotoxic']+ final['Pred_Tubulin']

print('Number data screen:', data.shape[0])
print('Number data cytotoxic predict', final[final['Cytotoxic']>0].shape[0])
print('Number data Tubulin predict', final[final['final_pre']>1].shape[0])

df.index = df['ID']
AD_pred = ad.data
AD_pred.index = AD_pred['ID']

data_pred = pd.concat([df[['ID','SMILES']], final.drop(final.columns[:2], axis =1),AD_pred.iloc[:,-3:]], axis =1)

sort_pred = data_pred.sort_values(by='Proba_Tubulin',ascending = False)
sort_pred 

/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


,ID,0,1,2,3,4,5,6,7,8,...,4090,4091,4092,4093,4094,4095,pChEMBL Value,AD_MDAMB231,AD_Tubulin,AD_A549
0,CHEMBL107,0,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,In,In,In
1,CHEMBL67,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,In,In,In
2,CHEMBL159,1,1,1,0,1,1,1,1,1,...,0,1,1,0,1,1,0,In,In,In
3,CHEMBL90555,1,1,1,0,1,1,1,1,1,...,0,1,1,0,1,1,0,In,In,In
4,CHEMBL553025,1,1,1,0,1,1,1,0,1,...,0,1,1,0,1,1,0,In,In,In


/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


The number of active structures: 6
Percentage of active structures: 66.66666666666667


/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


The number of active structures: 8
Percentage of active structures: 88.88888888888889
The number of active structures: 7
Percentage of active structures: 77.77777777777777
Number data screen: 9
Number data cytotoxic predict 9
Number data Tubulin predict 7


/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/tmp/ipykernel_201259/1596180079.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Cytotoxic'][final['Cytotoxic'] >0] =1


,ID,SMILES,Proba_A549,Pred_A549,Proba_MDAMB231,Pred_MDAMB231,Proba_Tubulin,Pred_Tubulin,Cytotoxic,final_pre,AD_MDAMB231,AD_Tubulin,AD_A549
CHEMBL107,CHEMBL107,COc1cc2c(c(OC)c1OC)-c1ccc(OC)c(=O)cc1[C@@H](NC...,98.681441,1,93.637892,1,90.037482,1,1,2,In,In,In
CHEMBL292702,CHEMBL292702,COc1cc2cc(c1Cl)N(C)C(=O)C[C@H](OC(=O)[C@H](C)N...,46.917301,0,81.717652,1,89.647886,1,1,2,In,In,In
CHEMBL92,CHEMBL92,CC(=O)O[C@@]12CO[C@@H]1C[C@H](O)[C@@]1(C)C(=O)...,48.040263,0,85.857290,1,83.037698,1,1,2,In,In,In
CHEMBL159,CHEMBL159,CC[C@]1(O)C[C@@H]2CN(CCc3c([nH]c4ccccc34)[C@@]...,59.189841,1,80.532852,1,74.703297,1,1,2,In,In,In
CHEMBL90555,CHEMBL90555,CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](...,61.454400,1,80.760927,1,74.703297,1,1,2,In,In,In
CHEMBL428647,CHEMBL428647,CC(=O)O[C@H]1C(=O)[C@@]2(C)[C@H]([C@H](OC(=O)c...,52.811456,1,85.348028,1,62.275631,1,1,2,In,In,In
CHEMBL553025,CHEMBL553025,CCC1=C[C@@H]2CN(C1)Cc1c([nH]c3ccccc13)[C@@](C(...,62.556691,1,75.156662,1,62.080845,1,1,2,In,In,In
CHEMBL1779811,CHEMBL1779811,C/C=C/C[C@H](C)[C@@H](OC)[C@@H](C)[C@H](O)C[C@...,25.389590,0,50.127023,1,49.200099,0,1,1,In,In,In
CHEMBL67,CHEMBL67,COc1ccc(/C=C\c2cc(OC)c(OC)c(OC)c2)cc1O,72.221395,1,49.782121,0,45.417407,0,1,1,In,In,In
